In [7]:
import tensorflow as tf
print(tf.__version__)

2.7.0


In [8]:
import tensorflow as tf
import tensorflow_hub as hub
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import re
import unicodedata
import nltk
#from transformers import pipeline
from nltk.corpus import stopwords
from tensorflow import keras
from tensorflow.keras.layers import Dense,Dropout, Input, BatchNormalization
from tqdm import tqdm
import pickle
from sklearn.metrics import confusion_matrix,f1_score,classification_report
import matplotlib.pyplot as plt
import itertools
from sklearn.utils import shuffle
from tensorflow.keras import regularizers
#from transformers import *
from transformers import BertTokenizer, TFBertModel, BertConfig,TFDistilBertModel,DistilBertTokenizer,DistilBertConfig
import pandas as pd
from transformers import AutoTokenizer, TFAutoModel
import numpy as np
import gc
import math
import json
import stanza
from tensorflow.keras import *
import tensorflow as tf
from tensorflow.keras import *
import tensorflow.keras.backend as K
from sklearn.model_selection import KFold
from sklearn.metrics import classification_report
from transformers import TFRobertaModel,RobertaTokenizer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.initializers import RandomUniform

from numpy.random import seed
import random as python_random
import os
import sys

np.random.seed(1)
python_random.seed(1)
tf.random.set_seed(1)

In [9]:
from datasets import load_dataset

# data = load_dataset("maneshkarun/median-3000")
data = load_dataset("maneshkarun/median3k_10000s")

Found cached dataset parquet (/home/ubuntu/.cache/huggingface/datasets/maneshkarun___parquet/maneshkarun--median3k_10000s-a12d2bed8c5e7733/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7)
100%|██████████| 1/1 [00:00<00:00, 669.48it/s]


In [10]:
train_data = data['train']

labels = train_data['bias'][:1051]

In [11]:
print(len(labels))

1051


In [12]:
lines = []
with open('summarized_usdb.txt', 'r') as file:
    for line in file:
        line = line.replace('--- ', '')
        lines.append(line.strip())

print(len(lines))

1051


In [13]:
with open('summarized_usdb.txt', encoding='utf-8') as f:
    contents = f.read()

records=contents.split("---")
records=records[1:]

In [14]:
len(records)

1051

In [15]:
summarized_data = pd.DataFrame(records,
               columns =['text'])

len_list = [len(ele.split()) for ele in records]
# len_list = [len(ele.split()) for ele in texts]
res = 0 if len(len_list) == 0 else (float(sum(len_list)) / len(len_list))
print("Average Length %s" % res) 

# temp_file = open("optimal_dataset_label.txt", "r")
#temp_file = open("labels_sc_279.txt", "r")

# data = temp_file.read()
  
# replacing end splitting the text 
# when newline ('\n') is seen.
# label_list = data.split("\n")
# print(label_list)
# label_list = label_list[0:-1]
# print(label_list)
# label_list = [int(i) for i in label_list]
# temp_file.close()
# label_list = temp_file.readlines()
# print(label_list)

summarized_data['label'] = labels

print(summarized_data)

Average Length 447.96194100856326
                                                   text  label
0       2017 Apr 1 10904 100 NA NA NANA NA NA N NA N...      0
1       Yr Ago Placed Placed Yr Average Placed yr Ag...      0
2       Yr Ago Placed Yr Average Yr Placed Marketed ...      0
3       Yr Ago Placed Yr Average Yr Placed yr Ago Ma...      0
4       The president is still one of his favorite T...      4
...                                                 ...    ...
1046    The coming year promises substantial growth ...      0
1047    Toronto is experiencing what brown people ha...      4
1048    Clark Hoyt the New York Times public editor ...      1
1049    In 1997 Deputy Prime Minister John Prescott ...      3
1050    The ruling elites in the U.S. have mounted a...      4

[1051 rows x 2 columns]


In [16]:
def create_model():
    inps = Input(shape = (max_len,), dtype='int64')
    masks= Input(shape = (max_len,), dtype='int64')
    dbert_layer = dbert_model(inps, attention_mask=masks)[0][:,0,:]
    dense_0 = Dense(512,activation='relu',kernel_regularizer=regularizers.l2(0.01))(dbert_layer)
    dropout_0= Dropout(0.5)(dense_0)
    pred = Dense(5, activation='softmax',kernel_regularizer=regularizers.l2(0.01))(dropout_0)
    model = tf.keras.Model(inputs=[inps,masks], outputs=pred)
    print(model.summary())
    return model   

In [17]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
total_accuracy=0
total_weighted_f1=0
total_micro_f1=0
total_weighted_precision=0
total_micro_precision=0
total_weighted_recall=0
total_micro_recall=0

for i in range(5):
  gc.collect()
  tf.keras.backend.clear_session()
  dbert_tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
  dbert_model = TFRobertaModel.from_pretrained('roberta-base')
  max_len=512
  sentences=summarized_data['text']
  labels=summarized_data['label']
  len(sentences),len(labels)
  model_0=create_model()
  input_ids=[]
  attention_masks=[]

  for sent in sentences:
    dbert_inps=dbert_tokenizer.encode_plus(sent,add_special_tokens = True,max_length =max_len,pad_to_max_length = True,return_attention_mask = True,truncation=True)
    input_ids.append(dbert_inps['input_ids'])
    attention_masks.append(dbert_inps['attention_mask'])
  input_ids=np.asarray(input_ids)

  attention_masks=np.array(attention_masks)
  labels=np.array(labels)
  train_inp,val_inp,train_label,val_label,train_mask,val_mask=train_test_split(input_ids,labels,attention_masks,test_size=0.1,random_state=42)
  log_dir='dbert_model'

  model_save_path='/home/ubuntu/HyperPartisan_Classification_Using_BERT/Summarization/roberta-summarization-512-'+str(i)+'-5labels.h5'

  loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
  accuracy = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')

  optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5)
  callbacks= [tf.keras.callbacks.ModelCheckpoint(filepath=model_save_path,monitor='val_accuracy',mode='max',save_best_only=True,save_weights_only=True),keras.callbacks.TensorBoard(log_dir=log_dir)]
  model_0.compile(loss=loss,optimizer=optimizer, metrics=[accuracy])
  gpu_info = !nvidia-smi
  gpu_info = '\n'.join(gpu_info)
  if gpu_info.find('failed') >= 0:
    print('Not connected to a GPU')
  else:
    print(gpu_info)
  history=model_0.fit([train_inp,train_mask],train_label,batch_size=8,epochs=5,validation_data=([val_inp,val_mask],val_label),callbacks=callbacks)
  pred_labels=[]

  model_saved= create_model()
  model_saved.compile(loss=loss,optimizer=optimizer, metrics=[accuracy])
  model_saved.load_weights('/home/ubuntu/HyperPartisan_Classification_Using_BERT/Summarization/roberta-summarization-512-'+str(i)+'-5labels.h5')

  for i in range(0,len(val_inp)):
    pred=model_saved.predict([val_inp[i].reshape(1,512),val_mask[i].reshape(1,512)])
    pred_label = pred.argmax(axis=1)
    pred_labels.append(pred_label)
  accuracy=accuracy_score(val_label, pred_labels)
  print("Accuracy: "+str(accuracy))
  total_accuracy=total_accuracy+accuracy
  
  weighted_f1=f1_score(val_label,pred_labels, average='weighted')
  print("Weighted F1: "+ str(weighted_f1))
  total_weighted_f1=total_weighted_f1+weighted_f1
  micro_f1=f1_score(val_label,pred_labels, average='micro')
  print("Micro F1: "+ str(micro_f1))
  total_micro_f1=total_micro_f1+micro_f1

  weighted_precision=precision_score(val_label, pred_labels, average='weighted')
  print("Weighted Precision: " + str(weighted_precision))
  total_weighted_precision=total_weighted_precision+weighted_precision
  micro_precision=precision_score(val_label, pred_labels, average='micro')
  print("Micro Precision: " + str(micro_precision))
  total_micro_precision=total_micro_precision+micro_precision

  weighted_recall=recall_score(val_label, pred_labels, average='weighted')
  print("Weighted Recall: " + str(weighted_recall))
  total_weighted_recall=total_weighted_recall+weighted_recall
  micro_recall=recall_score(val_label, pred_labels, average='micro')
  print("Micro Recall: " + str(micro_recall))
  total_micro_recall=total_micro_recall+micro_recall


2023-06-25 18:55:00.721021: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-06-25 18:55:00.725025: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-06-25 18:55:00.725154: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-06-25 18:55:00.725445: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

InternalError: Exception encountered when calling layer "query" (type Dense).

Attempting to perform BLAS operation using StreamExecutor without BLAS support [Op:MatMul]

Call arguments received:
  • inputs=tf.Tensor(shape=(1, 2, 768), dtype=float32)

In [ ]:

print("Average Accuracy: "+str(total_accuracy/5))
print("Average Weighted F1: "+str(total_weighted_f1/5))
print("Average Micro F1: "+str(total_micro_f1/5))
print("Average Weighted Precision: "+str(total_weighted_precision/5))
print("Average Micro Precision: "+str(total_micro_precision/5))
print("Average Weighted Recall: "+str(total_weighted_recall/5))
print("Average Micro Recall: "+str(total_micro_recall/5))